In [1]:
# code by Tae Hwan Jung @graykode
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor
# must have same length, or padding needed
sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict) # number of Vocabulary

# NNLM Parameter
n_step = 2 # n-1, input length
n_hidden = 2 # h

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        # word index to one hot encoding
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)
    
    return input_batch, target_batch

In [2]:

input_batch, target_batch = make_batch(sentences)

input_batch = Variable(torch.Tensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))
print(input_batch)

tensor([[[0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1.]]])
tensor([[0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.]])


tensor([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [6]:
print(input_batch[0])
print(input_batch[0].view(-1))

tensor([[0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.]])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])


In [3]:
# Model
# Y = b + X*W + tanh(d + X*H)*U
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        
        self.H = nn.Parameter(torch.randn(n_step * n_class, n_hidden).type(dtype))
        self.W = nn.Parameter(torch.randn(n_step * n_class, n_class).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))

    def forward(self, X):
        # concat all encoding
        input = X.view(-1, n_step * n_class) # [batch_size, n_step * n_class]
        tanh = torch.tanh(self.d + torch.mm(input, self.H)) # [batch_size, n_hidden]
        output = self.b + torch.mm(input, self.W) + torch.mm(tanh, self.U) # [batch_size, n_class]
        return output

In [4]:
model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Training
for epoch in range(5000):

    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

# Test
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 0.096678
Epoch: 2000 cost = 0.026012
Epoch: 3000 cost = 0.011057
Epoch: 4000 cost = 0.005576
Epoch: 5000 cost = 0.003052
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
